In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [6]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [11]:
sen_df = spark.createDataFrame([
    (0, 'I love learning spark'),
    (1, 'I would also like to learn scala'),
    (2, 'Yesterday the temperature was -7 celcius'),
    (3, 'Enthusiastic,innovative,happy,energetic,willing')
],['id','sentence'])

In [13]:
sen_df.show(truncate=False)

+---+-----------------------------------------------+
|id |sentence                                       |
+---+-----------------------------------------------+
|0  |I love learning spark                          |
|1  |I would also like to learn scala               |
|2  |Yesterday the temperature was -7 celcius       |
|3  |Enthusiastic,innovative,happy,energetic,willing|
+---+-----------------------------------------------+



In [14]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [16]:
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words', pattern='\\W')

In [17]:
count_tokens = udf(lambda words:len(words),IntegerType())

In [18]:
tokenized = tokenizer.transform(sen_df)

In [20]:
tokenized.show(truncate=False)

+---+-----------------------------------------------+-------------------------------------------------+
|id |sentence                                       |words                                            |
+---+-----------------------------------------------+-------------------------------------------------+
|0  |I love learning spark                          |[i, love, learning, spark]                       |
|1  |I would also like to learn scala               |[i, would, also, like, to, learn, scala]         |
|2  |Yesterday the temperature was -7 celcius       |[yesterday, the, temperature, was, -7, celcius]  |
|3  |Enthusiastic,innovative,happy,energetic,willing|[enthusiastic,innovative,happy,energetic,willing]|
+---+-----------------------------------------------+-------------------------------------------------+



In [21]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|I love learning s...|[i, love, learnin...|     4|
|  1|I would also like...|[i, would, also, ...|     7|
|  2|Yesterday the tem...|[yesterday, the, ...|     6|
|  3|Enthusiastic,inno...|[enthusiastic,inn...|     1|
+---+--------------------+--------------------+------+



In [22]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [24]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show(truncate=False)

+---+-----------------------------------------------+-----------------------------------------------------+------+
|id |sentence                                       |words                                                |tokens|
+---+-----------------------------------------------+-----------------------------------------------------+------+
|0  |I love learning spark                          |[i, love, learning, spark]                           |4     |
|1  |I would also like to learn scala               |[i, would, also, like, to, learn, scala]             |7     |
|2  |Yesterday the temperature was -7 celcius       |[yesterday, the, temperature, was, 7, celcius]       |6     |
|3  |Enthusiastic,innovative,happy,energetic,willing|[enthusiastic, innovative, happy, energetic, willing]|5     |
+---+-----------------------------------------------+-----------------------------------------------------+------+



In [25]:
from pyspark.ml.feature import StopWordsRemover

In [28]:
sentence_df = spark.createDataFrame([
    (0, ['I','will','pizza','tonight','with','Theo']),
    (1, ['The','sun','has','not','appeared','for','40','days'])
],['id','tokens'])

In [32]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [35]:
remover.transform(sentence_df).show(truncate=False)

+---+---------------------------------------------+-------------------------+
|id |tokens                                       |filtered                 |
+---+---------------------------------------------+-------------------------+
|0  |[I, will, pizza, tonight, with, Theo]        |[pizza, tonight, Theo]   |
|1  |[The, sun, has, not, appeared, for, 40, days]|[sun, appeared, 40, days]|
+---+---------------------------------------------+-------------------------+



In [36]:
from pyspark.ml.feature import NGram

In [40]:
word_df = spark.createDataFrame([
    (0, ['I','will','pizza','tonight','with','Theo']),
    (1, ['The','sun','has','not','appeared','for','40','days'])
],['id','words'])

In [41]:
ngram = NGram(n=2, inputCol='words',outputCol='grams')

In [44]:
ngram.transform(word_df).select('grams').show(truncate=False)

+------------------------------------------------------------------------+
|grams                                                                   |
+------------------------------------------------------------------------+
|[I will, will pizza, pizza tonight, tonight with, with Theo]            |
|[The sun, sun has, has not, not appeared, appeared for, for 40, 40 days]|
+------------------------------------------------------------------------+



In [45]:
sentence_df = spark.createDataFrame([
    (0, 'I love learning spark'),
    (1, 'I would also like to learn scala'),
    (2, 'Yesterday the temperature was -7 celcius'),
    (3, 'Enthusiastic,innovative,happy,energetic,willing')
],['id','sentence'])

In [46]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [47]:
words_data = tokenizer.transform(sentence_df)

In [48]:
words_data.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|I love learning s...|[i, love, learnin...|
|  1|I would also like...|[i, would, also, ...|
|  2|Yesterday the tem...|[yesterday, the, ...|
|  3|Enthusiastic,inno...|[enthusiastic,inn...|
+---+--------------------+--------------------+



In [55]:
from pyspark.ml.feature import HashingTF,IDF

In [51]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [52]:
feature_data = hashing_tf.transform(words_data)

In [54]:
feature_data.show()

+---+--------------------+--------------------+--------------------+
| id|            sentence|               words|         rawFeatures|
+---+--------------------+--------------------+--------------------+
|  0|I love learning s...|[i, love, learnin...|(262144,[24417,16...|
|  1|I would also like...|[i, would, also, ...|(262144,[24417,43...|
|  2|Yesterday the tem...|[yesterday, the, ...|(262144,[6781,255...|
|  3|Enthusiastic,inno...|[enthusiastic,inn...|(262144,[88758],[...|
+---+--------------------+--------------------+--------------------+



In [56]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [57]:
idf_model = idf.fit(feature_data)

In [59]:
rescaled_data = idf_model.transform(feature_data)

In [62]:
rescaled_data.select(['rawFeatures','features']).show(truncate=False)

+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rawFeatures                                                                           |features                                                                                                                                                                                       |
+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(262144,[24417,163984,186480,234657],[1.0,1.0,1.0,1.0])                               |(262144,[24417,163984,186480,234657],[0.5108256237659907,0.9162907318